# Notebook

In the Model learning step, the prepared dataset from [2_EDA](https://github.com/Rudinius/Bike_usage_Bremen/blob/57e21c8dd687aadc1498f82241cf662840c8b871/2_EDA.ipynb) is loaded. Then different machine learning algorithms are trained and compared to each other.

<a name="content"></a>
# Content 

* [1. Import libraries and mount drive](#1)
* [2. Import datasets](#2)
* [3. Transform columns](#3)
* [4. Establish baseline benchmark](#4)
* [5. Training machine learning algorithms](#5)
    * [5.2. XGBoost](#5.2.)
    * [5.3. Multilayer perceptron](#5.3.)
    * [5.4. Recurrent Neural Network](#5.4.)

<a name="1"></a>
# 1.&nbsp;Import libraries
[Content](#content)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import plotly.express as px
import random
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

<a name="2"></a>
#2.&nbsp;Import dataset
[Content](#content)

Next, we will import the processed dataset from [2_EDA](../Bike_usage_Bremen/2_EDA.ipynb).

In [ ]:
# Set base url
url = "https://raw.githubusercontent.com/Rudinius/Bike_usage_Bremen/main/data/"

In [ ]:
# Import dataset

# We will also parse the date column as datetime64 and set it to the index column
df = pd.read_csv(url + "03_training_data/" + "2023-04-02_df_full.csv", 
                         parse_dates=[0], index_col=[0])

# Check the correct loading of dataset
df.head()

,Weekday,GrafMoltkeStrEast,GrafMoltkeStrWest,HastedterBrStr,LangemarckStrEast,LangemarckStrWest,Osterdeich,RadwegKleineWeser,SchwachhauserRing,WachmannStrAusSouth,...,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,Holiday,Vacation
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1,261.0,290.0,381.0,312.0,308.0,870.0,410.0,391.0,514.0,...,9.1,6.9,0.0,233.0,19.4,50.4,1001.8,0,Neujahr,Weihnachtsferien
2013-01-02,2,750.0,876.0,1109.0,1258.0,1120.0,2169.0,1762.0,829.0,1786.0,...,7.1,1.8,0.0,246.0,20.2,40.0,1017.5,30,NaN,Weihnachtsferien
2013-01-03,3,931.0,1015.0,1603.0,1556.0,1480.0,2295.0,2287.0,1196.0,2412.0,...,10.6,0.9,0.0,257.0,23.8,45.7,1024.5,0,NaN,Weihnachtsferien
2013-01-04,4,500.0,587.0,1284.0,703.0,626.0,1640.0,1548.0,1418.0,964.0,...,9.7,0.0,0.0,276.0,25.2,48.2,1029.5,0,NaN,Weihnachtsferien
2013-01-05,5,1013.0,1011.0,1284.0,1856.0,1621.0,4128.0,4256.0,3075.0,2065.0,...,8.6,0.1,0.0,293.0,20.2,41.0,1029.9,0,NaN,Weihnachtsferien


<a name="3"></a>
# 3. Transform columns
[Content](#content)

We need to transform the columns `Holiday` and `Vacation` using `One-Hot-Encoding` to change the categorical columns to numerical columns. Then we need to drop the original columns.

In [ ]:
# Use One Hot Encoder
OH_encoder = OneHotEncoder()
transformed_array = OH_encoder.fit_transform(df.loc[:,"Holiday": "Vacation"]).toarray()
df_holiday_vacation_transformed = pd.DataFrame(transformed_array, 
                              columns=OH_encoder.get_feature_names_out(), 
                              index = df.index)              
# Drop the columns with Holiday_nan and Vacation_nan as those hold no additional value          
df_holiday_vacation_transformed = df_holiday_vacation_transformed.drop(["Holiday_nan", "Vacation_nan"], axis=1)

# Drop the old categorical columns Holiday and Vacation
df_transformed = df.drop(["Holiday", "Vacation"], axis=1)

# Add the new columns from OHE
df_transformed = pd.concat([df_transformed, df_holiday_vacation_transformed], axis=1)

We will add the year, month and day as seperate columns to give the algorithm the chance to pick up more granular and seasonal patterns.

In [ ]:
df_date = pd.DataFrame(data = {
    "Year": df.index.year,
    "Month": df.index.month,
    "Day": df.index.day
}, index=pd.to_datetime(df.index.values))
    
df_transformed_date = pd.concat([df_date, df_transformed], axis=1)

# Check dataframe
#df_transformed_date.head()

Now, after all those transformations, we have out final dataset, to train our machine learning algorithms on.

<a name="4"></a>
# 4. Establish baseline benchmark
[Content](#content)


For our current task of creating model to predict the amount of cyclers for a given day, we do not have any baseline accuracy to predict the value that we could use, to measure our model against. 
For this reason, we will create a naive baseline model. For this, we will simply predict the amount of a day based on the value of previous day.

In [ ]:
# Evaluate the model's performance using RMSE

# Select the `Total` column as our y_test and preds arrays
y_test = preds = df.loc[:,"Total"]

rmse = 0
length = y_test.shape[0]

# Loop from 0 to second last entry, as we can only use seconds last entry to
# predict the last entry of series
for i in range(length-1):
    # The mean_sqared_error function expects an array as input, therfore we 
    # concatenate the range from current value to current value + 1 (excluding)
    rmse += np.sqrt(mean_squared_error(y_test[i+1:i+2], preds[i:i+1]))

# Divide rmse value by number of pairs
rmse = rmse / (length-1)
print("RMSE: %f" % (rmse))

RMSE: 9083.225418


If we were naivly predicting the current value with the last value, we get an error over the entire dataset of approximately $9,111$. 

This is our naive benchmark to compare our model against. 

<a name="5"></a>
# 5. Training machine learning algorithms
[Content](#content)

<a name="5.2."></a>
## 5.2. XGBoost
[Content](#content)

Specifically for XGBoost algorithm, we will add the data points of the previous time steps as features to feature vector. For RNNs, this will probably not be necessary, as RNNs have built-in memory units that allow them to store information from previous steps.

In this case, we will only add the last 3 values, as the observed improvement of accuracy (RMSE score) is drastically decressing with each further time step added after 3 steps.

Improvements:
* 1 day 6291 2,8%
* 2 day 6157 2,1%
* 3 day 6120 0,6%

In [ ]:
# Create empty new dataframe
df_lagged_days = pd.DataFrame({})

# Select the number of lagged days
go_back_x_days = 3

for i in range(go_back_x_days):
    # Shift the values in "Total" by i and assign to new column prev_Total_i+1
    df_lagged_days[f'prev_Total_{i+1}'] = df_transformed_date['Total'].shift(i+1)

# Concat new dataframe with old dataframe
df_transformed_date_lagged = pd.concat([df_transformed_date, df_lagged_days], axis=1)

# Check output
#df_transformed_date_lagged.loc[:,["prev_Total_1", "prev_Total_2", "prev_Total_3","Total"]]

In [ ]:
# Load the data into a pandas dataframe
data = df_transformed_date_lagged

# Define the features and target
# Higly correlated features have been removed (tavg, tmax, wpgt)
features = ['Year', 'Month', 'Day', 'Weekday',
       'tmax', 'prcp', 'snow', 'wspd', 'pres', 'tsun',
       'Holiday_1. Weihnachtsfeiertag', 'Holiday_2. Weihnachtsfeiertag',
       'Holiday_Christi Himmelfahrt', 'Holiday_Karfreitag', 'Holiday_Neujahr',
       'Holiday_Ostermontag', 'Holiday_Pfingstmontag',
       'Holiday_Reformationstag', 'Holiday_Tag der Arbeit',
       'Holiday_Tag der Deutschen Einheit', 'Vacation_Herbstferien',
       'Vacation_Osterferien', 'Vacation_Pfingstferien',
       'Vacation_Sommerferien', 'Vacation_Weihnachtsferien',
       'Vacation_Winterferien', "prev_Total_1", "prev_Total_2", "prev_Total_3"]

target = 'Total'

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], 
                                                    test_size=0.2, shuffle=True, random_state=0)

Using `GridSearchCV` to select optimal parameters.

In [ ]:
params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [700, 1000, 1200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.1, 0.5, 1.0],
    'reg_lambda': [0.1, 0.5, 1.0],
    'min_child_weight': [1, 5, 10]
}

xg_reg = xgb.XGBRegressor(objective='reg:squarederror')

grid_search = GridSearchCV(xg_reg, param_grid=params, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

In [ ]:
# Build the XGBoost regressor model with selected hyper parameters
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 1.0, learning_rate = 0.01, 
                          max_depth = 12, n_estimators = 1000, reg_alpha = 0.1, reg_lambda = 1.0, 
                          subsample = 0.8, min_child_weight=5)

xg_reg.fit(X_train, y_train)

# Predict on the test set
preds = xg_reg.predict(X_test)

# Evaluate the model's performance using RMSE
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

* 02.04.23: 6468
* 10.04.23: 6120

As an alternative, instead of `train_test_split`, we try `TimeSeriesSplit` and compare the performance.

In [ ]:
# Split the data into training and test sets
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(data):
    X_train, X_test = data.iloc[train_index][features], data.iloc[test_index][features]
    y_train, y_test = data.iloc[train_index][target], data.iloc[test_index][target]

    xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=1.0, learning_rate=0.01,
                              max_depth=12, n_estimators=1000, reg_alpha=0.1, reg_lambda=1.0,
                              subsample=0.8, min_child_weight=5)

    xg_reg.fit(X_train, y_train)

    # Predict on the test set
    preds = xg_reg.predict(X_test)

    # Evaluate the model's performance using RMSE
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print("RMSE: %f" % (rmse))


`train_test_split` has a better performance on the accuracy of the model than `TimeSeriesSplit`.

Plot random different data points to analyze big differences between prediction and actual value in more detail.

In [ ]:
# Select 100 random datapoints from y_test
random_indices = random.sample(range(len(y_test)), 100)
y_test_sample = y_test[random_indices]
preds_sample = preds[random_indices]

# Create plot using plotly express
fig = px.scatter()
fig.add_scatter(x=y_test_sample.index, y=y_test_sample, mode='markers', name='y_test', marker=dict(color='blue'))
fig.add_scatter(x=y_test_sample.index, y=preds_sample, mode='markers', name='preds', marker=dict(color='red'))
fig.update_layout(title='y_test vs. preds, random values', xaxis_title='Date', yaxis_title='Values')
fig.show()

<a name="5.3."></a>
## 5.3. Multilayer Perceptron
[Content](#content)

<a name="5.4."></a>
## 5.4. Recurrent Neural Network
[Content](#content)

In [ ]:
print(df_transformed_date_lagged.isna().sum())
#print(X_train.shape[0], X_train.shape[1])
#print(type(X_train))

Year                                 0
Month                                0
Day                                  0
Weekday                              0
GrafMoltkeStrEast                    0
GrafMoltkeStrWest                    0
HastedterBrStr                       0
LangemarckStrEast                    0
LangemarckStrWest                    0
Osterdeich                           0
RadwegKleineWeser                    0
SchwachhauserRing                    0
WachmannStrAusSouth                  0
WachmannStrEinNorth                  0
WilhelmKaisenBrEast                  0
WilhelmKaisenBrWest                  0
Total                                0
tavg                                 0
tmin                                 0
tmax                                 0
prcp                                 0
snow                                 0
wdir                                 0
wspd                                 0
wpgt                                 0
pres                     

In [ ]:
# Load the data into a pandas dataframe
data = df_transformed_date

# Define the features and target
# Higly correlated features have been removed (tavg, tmax, wpgt)
features = ['Year', 'Month', 'Day', 'Weekday',
       'tmax', 'prcp', 'snow', 'wspd', 'pres', 'tsun',
       'Holiday_1. Weihnachtsfeiertag', 'Holiday_2. Weihnachtsfeiertag',
       'Holiday_Christi Himmelfahrt', 'Holiday_Karfreitag', 'Holiday_Neujahr',
       'Holiday_Ostermontag', 'Holiday_Pfingstmontag',
       'Holiday_Reformationstag', 'Holiday_Tag der Arbeit',
       'Holiday_Tag der Deutschen Einheit', 'Vacation_Herbstferien',
       'Vacation_Osterferien', 'Vacation_Pfingstferien',
       'Vacation_Sommerferien', 'Vacation_Weihnachtsferien',
       'Vacation_Winterferien']

target = 'Total'

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], 
                                                    test_size=0.2, shuffle=True, random_state=0)

In [ ]:

import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

# Assuming X_train, X_test, y_train, y_test are already defined and contain the appropriate data




num_samples = X_train.shape[0]
# Build the LSTM model
"""
model = Sequential()
model.add(LSTM(units=256, input_shape=(1, X_train.shape[1]), activation='relu'))
# Add a dense output layer with a single unit (for regression) and no activation function
model.add(Dense(units=1))
"""
"""
model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(64),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])
"""


# Build the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(256, activation='relu'),
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(128, activation='relu'),
    # Add another LSTM layer with 128 units and ReLU activation
    tf.keras.layers.LSTM(64, activation='relu'),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

X_train_3d = np.reshape(X_train.to_numpy(), (num_samples, 1, X_train.shape[1]))  # Fix variable name
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=Adam(learning_rate=0.001),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

# Train the model
model.fit(X_train_3d, y_train, epochs=100, batch_size=64, verbose=1)

Epoch 1/100


ValueError: ignored

In [ ]:
# Predict on the test set
#X_test_3d = np.reshape(X_test.to_numpy(), (X_test.shape[0], 1, X_test.shape[1]))  # Reshape X_test

num_samples = X_test.shape[0]
X_test_3d = np.reshape(X_test.to_numpy(), (num_samples, 1, X_test.shape[1]))  # Fix variable name
preds = model.predict(X_test_3d)

# Evaluate the model's performance using RMSE
rmse = np.sqrt(mean_squared_error(y_test, np.reshape(preds, (y_test.shape[0]))))
print("RMSE: %f" % (rmse))


23/23 [==============================] - 0s 6ms/step
2018-03-11    26957.0
2014-07-27    37410.0
2016-09-30    39593.0
2022-06-05    50288.0
2017-09-29    41285.0
               ...   
2013-08-06    37690.0
2014-11-02    29683.0
2020-06-27    33080.0
2016-08-31    53149.0
2022-01-16    12165.0
Name: Total, Length: 731, dtype: float64
[30535.756 37923.53  32049.592 36079.492 31525.645 28276.818 31182.781
 28032.14  26946.822 35099.41  40275.06  31049.348 38569.285 31857.996
 30947.023 28566.012 28895.145 29982.74  36054.23  28999.047 29830.938
 30775.758 27452.115 31243.29  27236.404 27673.469 29084.553 30560.8
 30556.438 34519.117 30605.236 31264.998 33066.836 20967.12  37602.754
 32578.305 39709.023 29811.994 28410.668 33195.67  27754.33  31554.383
 27051.059 28399.223 35825.227 27933.12  39415.55  35235.44  35060.434
 32877.6   31883.713 31583.65  28782.07  34868.4   32103.193 28370.572
 26299.674 29966.117 36611.188 40976.992 27086.205 35797.164 28439.223
 33622.242 28705.676 38175.